In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Spark HW Local") \
    .getOrCreate()

In [3]:
# The output of spark.version.
print(f'Spark version is {spark.version}')

Spark version is 3.4.2


In [4]:
df_spark = spark.read \
    .option('header', 'true') \
    .option('inferschema', 'true') \
    .csv('data/raw/fhv_tripdata_2019-10.csv')

In [5]:
# df_spark.printSchema()

In [6]:
# df_spark.schema

In [7]:
'''
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)
])
'''

"\nschema = types.StructType([\n    types.StructField('dispatching_base_num', types.StringType(), True), \n    types.StructField('pickup_datetime', types.TimestampType(), True), \n    types.StructField('dropOff_datetime', types.TimestampType(), True), \n    types.StructField('PUlocationID', types.IntegerType(), True), \n    types.StructField('DOlocationID', types.IntegerType(), True), \n    types.StructField('SR_Flag', types.StringType(), True), \n    types.StructField('Affiliated_base_number', types.StringType(), True)\n])\n"

In [8]:
"""
df_spark = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("data/raw/fhv_tripdata_2019-10.csv")
"""

'\ndf_spark = spark.read     .option("header", "true")     .schema(schema)     .csv("data/raw/fhv_tripdata_2019-10.csv")\n'

In [9]:
df_spark = df_spark.repartition(6)

In [10]:
df_spark.write.parquet('data/output/fhv10/', mode='overwrite')

In [11]:
df_spark.createOrReplaceTempView('fhv_tripdata')

In [12]:
# Number of trips on the 15th of October

print("Number of trips on the 15th of October:")
spark.sql(
"""
SELECT
    COUNT(1) AS cctober_15_trips
FROM
    fhv_tripdata
WHERE
    DATE(pickup_datetime)='2019-10-15'
"""
).show()

Number of trips on the 15th of October:
+----------------+
|cctober_15_trips|
+----------------+
|           62610|
+----------------+



In [13]:
# The longest trip in the dataset in hours
# TIMESTAMPDIFF(HOUR, ... also works, but it returns 0 for the value after the decimal point

print("The longest trip in the dataset in hours:")
spark.sql(
"""
SELECT
    TIMESTAMPDIFF(SECOND, pickup_datetime, dropOff_datetime)/3600.0 AS longest_trip
FROM
    fhv_tripdata
ORDER BY
    longest_trip DESC
LIMIT 1
"""
).show()

The longest trip in the dataset in hours:
+-------------+
| longest_trip|
+-------------+
|631152.500000|
+-------------+



In [14]:
df_zones= spark.read.parquet('data/zones/')

In [15]:
df_zones.createOrReplaceTempView('zones')

In [16]:
# Least frequent pickup location zone

print("The five Least frequent pickup location zones:")
df_least_zones = spark.sql(
"""
SELECT
    COUNT(1) AS trips_per_pickup_zone,
    zones.Zone AS pickup_zone
FROM
    fhv_tripdata
JOIN
    zones ON fhv_tripdata.PUlocationID = zones.LocationID
GROUP BY
    zones.Zone
ORDER BY
    trips_per_pickup_zone ASC
LIMIT 5;
""" 
).show()

The five Least frequent pickup location zones:
+---------------------+--------------------+
|trips_per_pickup_zone|         pickup_zone|
+---------------------+--------------------+
|                    1|         Jamaica Bay|
|                    2|Governor's Island...|
|                    5| Green-Wood Cemetery|
|                    8|       Broad Channel|
|                   14|     Highbridge Park|
+---------------------+--------------------+

